In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm

from quapopt.additional_packages.ancillary_functions_usra import ancillary_functions as anf
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)
from quapopt.hamiltonians.generators import build_hamiltonian_generator

In [ ]:


#TODO(FBM): add description and refactor arguments

hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick

full_noq_range = list(range(4, 26)) + list(range(30, 51, 10))


#full_noq_range += [100*i for i in range(2, 11)]
# full_noq_range += [2 ** i for i in range(5, 13)]
full_noq_range = [int(i*(i-1)/2) for i in full_noq_range]

full_noq_range = [128,256,512]


full_noq_range = [129]
full_noq_range = sorted(list(set(full_noq_range)))

full_seeds_range = list(range(0, 1))

coefficients_type = COEFFICIENTS_TYPE.DISCRETE
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
# _coefficients_distribution_properties = 1
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}

coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

arguments_dict_builder = {}

localities = None
if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
    localities = (1,2)
elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
    localities = (2,)
elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
    localities = (1, 2)
elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.Wishart]:
    pass

generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

print(len(full_noq_range),
      len(full_seeds_range),
      len(full_noq_range) * len(full_seeds_range) * 120 / 60 / 60)

print(full_noq_range)


In [ ]:

solver_kwargs = {'solver_name': "BURER2002",
                              'solver_timeout': 120}
both_directions = True


for number_of_qubits in tqdm(full_noq_range):
    anf.cool_print("NUMBER OF QUBITS", number_of_qubits)

    for seed_hamiltonian in full_seeds_range:
        print("INSTANCE:", seed_hamiltonian)

        arguments_instance = {'number_of_qubits': number_of_qubits,
                              'seed': seed_hamiltonian,
                              'read_from_drive_if_present': True}
        if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
            pass
        elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
            graph_degree = 3
            arguments_instance['graph_degree'] = graph_degree
        elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
            average_degree = number_of_qubits // 2
            edge_probability = average_degree / (number_of_qubits - 1)
            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
            average_degree = number_of_qubits // 2
            edge_probability = average_degree / (number_of_qubits - 1)
            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
            clause_density = 1.1
            arguments_instance['clause_density'] = clause_density

        elif hamiltonian_model in [HAMILTONIAN_MODELS.WishartPlantedEnsemble]:
            pass

        cost_hamiltonian = generator_cost_hamiltonian.generate_instance(**arguments_instance)

        #This solving is required to get lowest energy state and highest energy state
        cost_hamiltonian.solve_hamiltonian(both_directions=both_directions,
                                           solver_kwargs=solver_kwargs)

        #This will write known solutions if they already exist in instance attributes (if not, it doesn't do anything)
        cost_hamiltonian.write_solutions_to_file()

        ground_state_energy = cost_hamiltonian.ground_state_energy
        highest_energy = cost_hamiltonian.highest_energy



In [ ]:
print("Okay, then")